Import libraries

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt


Data Loading , Intervals declreation , filter

In [ ]:
# Load the dataset
df1 = pd.read_csv(r'C:\Users\delta\Desktop\male.csv')  # Male data
df2 = pd.read_csv(r'C:\Users\delta\Desktop\female.csv')  # Female data

# Convert columns to numeric in case they are read as strings
df1 = df1.apply(pd.to_numeric, errors='coerce')
df2 = df2.apply(pd.to_numeric, errors='coerce')

# Extract time and EMG values for both subjects (male and female)
time_male = df1['Data Set 1:Time(s)']
time_female = df2['Data Set 2:Time(s)']  # Ensure time alignment
emg_male = df1['Data Set 1:EMG Rectified(mV)']
emg_female = df2['Data Set 2:EMG Rectified(mV)']

# Define a Butterworth low-pass filter to remove noise
def butter_lowpass_filter(data, cutoff, fs, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

# Set parameters for the low-pass filter
fs = 100  # Sampling rate in Hz (assuming 100 Hz)
cutoff = 20  # Cutoff frequency in Hz, adjusted to preserve more signal

# Apply the low-pass filter to the EMG signals
emg_male_filtered = butter_lowpass_filter(emg_male, cutoff, fs)
emg_female_filtered = butter_lowpass_filter(emg_female, cutoff, fs)

# Define time intervals for different phases
intervals = {
    "no_movement": (0, 5),
    "contraction_1": (5, 10),
    "relaxation_1": (10, 15),
    "contraction_2": (15, 20),
    "relaxation_2": (20, 25)
}

# Function to filter EMG data within a specific time interval
def filter_interval(time, emg, interval):
    start, end = interval
    mask = (time >= start) & (time < end)
    return emg[mask]

# Store EMG data for each phase (male and female)
emg_data = {
    "male": {},
    "female": {}
}


Plotting ad analysis

In [ ]:
# Extract EMG data for each phase for both subjects
for phase, interval in intervals.items():
    emg_data['male'][phase] = filter_interval(time_male, emg_male_filtered, interval)
    emg_data['female'][phase] = filter_interval(time_female, emg_female_filtered, interval)

    # Remove NaN values from the filtered data
    emg_data['male'][phase] = emg_data['male'][phase][~pd.isna(emg_data['male'][phase])]
    emg_data['female'][phase] = emg_data['female'][phase][~pd.isna(emg_data['female'][phase])]

    # Debugging output to check the contents of filtered data
    print(f"{phase} - Male data:", emg_data['male'][phase])
    print(f"{phase} - Female data:", emg_data['female'][phase])

# Plotting histograms for male and female contraction and relaxation phases
plt.figure(figsize=(12, 8))

# Histogram for Male - Contraction vs Relaxation
plt.subplot(2, 2, 1)
if emg_data['male']['contraction_1'].size > 0:
    plt.hist(emg_data['male']['contraction_1'], bins=20, alpha=0.7, label='Contraction 1')
if emg_data['male']['relaxation_1'].size > 0:
    plt.hist(emg_data['male']['relaxation_1'], bins=20, alpha=0.7, label='Relaxation 1', color='orange')
plt.title("Male - Contraction vs Relaxation (Phase 1)")
plt.xlabel("EMG Rectified (mV)")
plt.ylabel("Frequency")
plt.legend()

plt.subplot(2, 2, 2)
if emg_data['male']['contraction_2'].size > 0:
    plt.hist(emg_data['male']['contraction_2'], bins=20, alpha=0.7, label='Contraction 2')
if emg_data['male']['relaxation_2'].size > 0:
    plt.hist(emg_data['male']['relaxation_2'], bins=20, alpha=0.7, label='Relaxation 2', color='orange')
plt.title("Male - Contraction vs Relaxation (Phase 2)")
plt.xlabel("EMG Rectified (mV)")
plt.ylabel("Frequency")
plt.legend()

# Histogram for Female - Contraction vs Relaxation
plt.subplot(2, 2, 3)
if emg_data['female']['contraction_1'].size > 0:
    plt.hist(emg_data['female']['contraction_1'], bins=20, alpha=0.7, label='Contraction 1')
if emg_data['female']['relaxation_1'].size > 0:
    plt.hist(emg_data['female']['relaxation_1'], bins=20, alpha=0.7, label='Relaxation 1', color='orange')
plt.title("Female - Contraction vs Relaxation (Phase 1)")
plt.xlabel("EMG Rectified (mV)")
plt.ylabel("Frequency")
plt.legend()

plt.subplot(2, 2, 4)
if emg_data['female']['contraction_2'].size > 0:
    plt.hist(emg_data['female']['contraction_2'], bins=20, alpha=0.7, label='Contraction 2')
if emg_data['female']['relaxation_2'].size > 0:
    plt.hist(emg_data['female']['relaxation_2'], bins=20, alpha=0.7, label='Relaxation 2', color='orange')
plt.title("Female - Contraction vs Relaxation (Phase 2)")
plt.xlabel("EMG Rectified (mV)")
plt.ylabel("Frequency")
plt.legend()

plt.tight_layout()
plt.show()